In [1]:
import pytest
import unittest
import numpy as np
from mne.filter import filter_data, resample
from yasa.main import (_corr, _covar, _rms, moving_transform, stft_power,
                       _index_to_events, get_bool_vector, trimbothstd,
                       _merge_close, spindles_detect, spindles_detect_multi)

# Load data
data = np.loadtxt('data_N2_spindles_15sec_200Hz.txt')
sf = 200
data_sigma = filter_data(data, sf, 12, 15, method='fir', verbose=0)

# Resample the data to 128 Hz
fac = 128 / sf
data_128 = resample(data, up=fac, down=1.0, npad='auto', axis=-1,
                    window='boxcar', n_jobs=1, pad='reflect_limited',
                    verbose=False)
sf_128 = 128


data_n3 = np.loadtxt('data_N3_no-spindles_30sec_100Hz.txt')
sf_n3 = 100

file_full = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data_full = file_full.get('data')
chan_full = file_full.get('chan')
sf_full = 100

# Load the hypnogram
hypno_full = np.load('data_full_6hrs_100Hz_hypno.npz').get('hypno')

## Downsampling

In [2]:
sp = spindles_detect(data, sf)
sp_no_ds = spindles_detect(data, sf, downsample=False)

display(sp)
display(sp_no_ds)

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry
0,3.32,4.06,0.74,81.800396,19.650618,2.722480,0.495178,12.859197,10.0,0.653333
1,13.26,13.85,0.59,99.302635,24.495293,2.827413,0.240493,12.158956,7.0,0.250000


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry
0,3.24,4.060,0.820,82.372425,19.130129,2.684425,0.480513,12.850270,11.0,0.696970
1,13.06,13.845,0.785,102.758017,23.991812,2.830781,0.226470,12.242634,10.0,0.443038


### What is the effect of downsampling on computation time?

In [3]:
%timeit -r 3 spindles_detect(data, sf)
%timeit -r 3 spindles_detect(data, sf, downsample=False)

7.14 ms ± 48.1 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)
10.5 ms ± 372 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [4]:
data_full_400Hz = resample(data_full, up=4, down=1.0, npad='auto', axis=-1, verbose=False)
print(data_full.shape, data_full_400Hz.shape)

(3, 2161058) (3, 8644232)


In [5]:
%timeit -r 3 spindles_detect(data_full[0, :], 100)  # No downsampling, sf = 100 Hz
%timeit -r 3 spindles_detect(data_full_400Hz[0, :], 400, downsample=True)  # Downsampling from 400 Hz to 100 Hz
%timeit -r 3 spindles_detect(data_full_400Hz[0, :], 400, downsample=False)  # No downsampling, sf = 400 Hz

2.57 s ± 28.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
2.62 s ± 92.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
7.14 s ± 65.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [6]:
# Multi-channel
%timeit -r 3 spindles_detect_multi(data_full_400Hz, 400, ch_names=chan_full, downsample=True)  # Downsampling from 400 Hz to 100 Hz
%timeit -r 3 spindles_detect_multi(data_full_400Hz, 400, ch_names=chan_full, downsample=False)  # No downsampling, sf = 400 Hz

8.03 s ± 206 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
20.8 s ± 74 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Hypnogram

In [7]:
# Hypnogram with sf = 128 (downsampling is not possible)
spindles_detect(data_128, sf_128, hypno=np.ones(data_128.size))

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Stage
0,3.3125,4.046875,0.734375,82.171575,19.554141,2.719616,0.499409,12.853133,10.0,0.673684,1
1,13.2500,13.835938,0.585938,98.708509,24.555503,2.829664,0.239602,12.166419,7.0,0.263158,1


In [8]:
# Hypnogram with only one unique value
sp = spindles_detect(data, sf, hypno=np.zeros(data.size))

14-Aug-19 16:38:00 | ERROR | None of the stages specified in `include` are present in hypno. Returning None.
